In [ ]:
import os
import pandas as pd
import tushare as ts
import numpy as np
from dateutil.relativedelta import relativedelta
import datetime
import matplotlib.pyplot as plt
import time
import calendar
ts.set_token('4264bdda6fbb66c87e086cfadaecdc887566b8c689f000965db52f3e')
pro = ts.pro_api()

# Get all SHSZ stock basic info from Tushare
stockinfo = pro.stock_basic(exchange='', list_status='L', fields='ts_code,fullname')
stockcode = stockinfo['ts_code'].tolist()

In [ ]:
# Define a function to input a date then return the date's quarter last day
def getCalendarDate(trddt):
    if type(trddt) == int:
        trddt = str(trddt)
    else:
        trddt = trddt
    if 0 < int(trddt[-4:]) <= 331:
        output = eval(trddt[0:4]) * 10000 + 331
    elif 401 <= int(trddt[-4:]) <= 630:
        output = eval(trddt[0:4]) * 10000 + 630
    elif 701 <= int(trddt[-4:]) <= 930:
        output = eval(trddt[0:4]) * 10000 + 930
    else:
        output = eval(trddt[0:4]) * 10000 + 1231
    return output

# Define another function to input a date then return the date's last quarter last day and a year before date
def getLastCalendar(trddt):
    if type(trddt) == int:
        trddt = str(trddt)
    else:
        trddt = trddt
    if 0 < int(trddt[-4:]) <= 331:
        output = (eval(trddt[0:4])-1) * 10000 + 1231
        output2 = (eval(trddt[0:4])-2) * 10000 + 1231
    elif 401 <= int(trddt[-4:]) <= 630:
        output = eval(trddt[0:4]) * 10000 + 331
        output2 = (eval(trddt[0:4])-1) * 10000 + 331
    elif 701 <= int(trddt[-4:]) <= 930:
        output = eval(trddt[0:4]) * 10000 + 630
        output2 = (eval(trddt[0:4])-1) * 10000 + 630
    else:
        output = eval(trddt[0:4]) * 10000 + 930
        output2 = (eval(trddt[0:4])-1) * 10000 + 930
    return [output, output2]

# Define a function to get monthlydate
def getMonthLast(trddt):
    # Return date format in 'YYYY-MM-DD'
    newdate = datetime.datetime.strptime(str(int(trddt)),'%Y%m%d').strftime('%Y-%m-%d')
    dateformat = datetime.date(*map(int, newdate.split('-')))
    # Get trading date's month start date
    this_month_start = datetime.datetime(dateformat.year, dateformat.month, 1)
    # Get trading date's month end date
    this_month_end = datetime.datetime(dateformat.year, dateformat.month, calendar.monthrange(dateformat.year, dateformat.month)[1])
    # Get trading date's last month end date
    last_month_end = this_month_start - datetime.timedelta(days=1)
    # Get trading date's month's three month end date
    three_month_end = this_month_start - relativedelta(months=2) - datetime.timedelta(days=1)
    # Get trading dates's month's half year end date
    six_month_end = this_month_start - relativedelta(months=5) - datetime.timedelta(days=1)
    # Get trading date's month's one year end date
    last_year_end = this_month_start - relativedelta(months=11) - datetime.timedelta(days=1)
    # Change into YYYYMMDD format
    thismonthend = eval(datetime.datetime.strptime(str(this_month_end.date()), '%Y-%m-%d').strftime('%Y%m%d'))
    lastmonthend = eval(datetime.datetime.strptime(str(last_month_end.date()), '%Y-%m-%d').strftime('%Y%m%d'))
    threemonthend = eval(datetime.datetime.strptime(str(three_month_end.date()), '%Y-%m-%d').strftime('%Y%m%d'))
    sixmonthend = eval(datetime.datetime.strptime(str(six_month_end.date()), '%Y-%m-%d').strftime('%Y%m%d'))
    lastyearend = eval(datetime.datetime.strptime(str(last_year_end.date()), '%Y-%m-%d').strftime('%Y%m%d'))
    return [thismonthend, lastmonthend, threemonthend, sixmonthend, lastyearend]

### Risk-free Rate (Shibor in %)

In [ ]:
os.chdir('Default_Data/Shibor/')
rf_mean = []
for file in os.listdir():
    print(file)
    data = pd.read_excel(file)
    year = file[7:11]
    changedate = []
    for j in data['日期'].tolist():
        formatdate = time.strftime("%Y%m%d",time.strptime(str(j)[0:19], "%Y-%m-%d %H:%M:%S"))
        changedate.append(getMonthLast(formatdate)[0])
    data['formatdate'] = pd.DataFrame(changedate)
    monthdate = list(set(changedate))
    for da in monthdate:
        eachmonth = data[data['formatdate'] == da]
        rf_mean.append(pd.DataFrame([da, np.mean(eachmonth['1Y'])/100]).T)
df_rf = pd.concat(rf_mean)
df_rf.columns = ['CalendarMonth','rf']
df_rfreset = df_rf.sort_values(by=['CalendarMonth'], ascending=True).reset_index().drop(['index'], axis=1)
rfuse = df_rfreset.drop(df_rfreset[df_rfreset['CalendarMonth'] >= 20210930].index)
os.chdir('../../')

### Then start to get each variables' raw data and put them together

In [ ]:
debtframe = []
bpsframe = []
for i in range(len(stockcode)):
    print(i)
    # 短期负债（流动负债合计）、长期负债（非流动负债合计）、企业债务面值（负债合计）
    dfdebt = pro.balancesheet(ts_code=stockcode[i], start_date='20090630', end_date='20210831', fields='ts_code,ann_date,f_ann_date,end_date,report_type,total_cur_liab,total_ncl,total_liab')
    if (dfdebt is None) == True:
        continue
    else:
        if dfdebt.empty == True:
            continue
        else:
            debtcalendar = []
            for j in range(dfdebt.shape[0]):
                debtcalendar.append(getMonthLast(dfdebt.iloc[j,2])[0])
            dfdebt['CalendarMonth'] = pd.DataFrame(debtcalendar)
            dfdebtdrop = dfdebt.drop_duplicates(subset=['ts_code','CalendarMonth'], keep='first',inplace=False).reset_index().drop(['index'], axis=1)
            dfdebtrf = dfdebtdrop.merge(rfuse, on='CalendarMonth', how='right').fillna(method='ffill')
            dfdebtrf['ts_code'] = stockcode[i]
            dfdebtrf['total_cur_liab'].fillna(0, inplace=True)
            dfdebtrf['total_ncl'].fillna(0, inplace=True)
            dfdebtrf['total_liab'].fillna(0, inplace=True)
            dfdebtfinal = dfdebtrf[dfdebtrf['CalendarMonth'] >= 20110101].reset_index().drop(['index'],axis=1)
            debtframe.append(dfdebtfinal)
    # 每股净资产
    dfbps = pro.fina_indicator(ts_code=stockcode[i], start_date='20090630', end_date='20210831', fields='ts_code,ann_date,end_date,bps')
    if (dfbps is None) == True:
        continue
    else:
        if dfbps.empty == True:
            continue
        else:
            bpscalendar = []
            for k in range(dfbps.shape[0]):
                if dfbps.iloc[k,1] == None:
                    bpscalendar.append(None)
                else:
                    bpscalendar.append(getMonthLast(dfbps.iloc[k,1])[0])
            dfbps['CalendarMonth'] = pd.DataFrame(bpscalendar)
            dfbpsdrop = dfbps.drop_duplicates(subset=['ts_code','CalendarMonth'], keep='first',inplace=False).reset_index().drop(['index'],axis=1)
            dfbpsbefore = dfbpsdrop.dropna(subset=['CalendarMonth'], inplace=False).reset_index().drop(['index'], axis=1)
            dfbpsrf = dfbpsbefore.merge(rfuse, on='CalendarMonth', how='right').fillna(method='ffill')
            dfbpsrf['ts_code'] = stockcode[i]
            dfbpsrf['bps'].fillna(0, inplace=True)
            dfbpsfinal = dfbpsrf[dfbpsrf['CalendarMonth'] >= 20110101].reset_index().drop(['index'],axis=1)
            bpsframe.append(dfbpsfinal)    
debtuse = pd.concat(debtframe).reset_index().drop(['index'], axis=1)
bpsuse = pd.concat(bpsframe).reset_index().drop(['index'], axis=1)

### 流通股数量、非流通股数量（Get from 'TRD_Capchg.csv'）

In [ ]:
os.chdir('Default_Data/')
shareoutstand = pd.read_csv('TRD_Capchg.csv')
os.chdir('../')
sharedata = shareoutstand[['Stkcd','Shrchgdt','Nshrttl','Nshra']]
sharedata['Stkcd'] = pd.DataFrame([str(x).zfill(6) for x in sharedata['Stkcd'].tolist()])
stkcd = sharedata['Stkcd'].tolist()
for i in range(len(stkcd)):
    if stkcd[i][0] == '0' or stkcd[i][0] == '3':
        stkcd[i] = stkcd[i] + '.SZ'
    else:
        stkcd[i] = stkcd[i] + '.SH'
sharedata['Stkcd'] = pd.DataFrame(stkcd)
sharedata['Shrchgdt'] = pd.DataFrame([x.replace('-','') for x in sharedata['Shrchgdt'].tolist()])
sharecalendar = []
for j in sharedata['Shrchgdt'].tolist():
    sharecalendar.append(getMonthLast(j)[0])
sharedata['CalendarMonth'] = pd.DataFrame(sharecalendar)
sharedrop = sharedata.drop_duplicates(subset=['Stkcd','CalendarMonth'], keep='last',inplace=False).reset_index().drop(['index'], axis=1)
shareframe = []
for code in list(set(sharedrop['Stkcd'].tolist())):
    eachstock = sharedrop[sharedrop['Stkcd'] == code].reset_index().drop(['index'], axis=1)
    eachmerge = eachstock.merge(rfuse, on='CalendarMonth', how='right').fillna(method='ffill')
    finaluse = eachmerge[eachmerge['CalendarMonth'] >= 20110101].reset_index().drop(['index'],axis=1)
    shareframe.append(finaluse)
shareuse = pd.concat(shareframe).reset_index().drop(['index'],axis=1)

In [ ]:
os.getcwd()

In [ ]:
# Get Calendar Month list
calendardate = rfuse['CalendarMonth'].tolist()[12:]

### Start to calculate KMV

In [ ]:
# Define nan
a = debtuse[debtuse['ts_code'] == '000001.SZ'][debtuse['CalendarMonth'] == 20210831]
nan = a.iloc[0,5]

In [ ]:
# function to calculate sigmaE
def sigmaE(n, squaresum, sumsquare):
    if (n <= 1) or (squaresum == None) or (sumsquare == None):
        sigmae = None
    else:
        if (squaresum/(n-1)) - sumsquare/(n*(n-1)) <= 0:
            sigmae = None
        else:
            nominator = math.sqrt((squaresum/(n-1)) - sumsquare/(n*(n-1)))
            denominator = math.sqrt(1/n)
            sigmae = nominator/denominator
    return sigmae

#import package
from scipy.optimize import fsolve
import math
from scipy import stats
def N(x):
    nx=stats.norm.cdf(x)
    return nx

#def calculate the V and sigmaV
def f(Equity,SigmaE,D,r,t=1):
    Equity=float(Equity)
    D=float(D)
    r=float(r)
    if Equity==0 or SigmaE==0 or D==0:
        result=[None,None]
        return result
    else:
        def ff(x):

            x0=float(x[0])
            x1=float(x[1])
                       
            d1=(math.log(abs(x0/D))+(r+0.5*(x1**2))*t)/(x1*math.sqrt(t))
            d2=d1-x1*math.sqrt(t)
            return[
                Equity-x0*N(d1)+D*math.exp(-r*t)*N(d2),
                SigmaE-x0*x1*(N(d1)/Equity)
            ]
        result=fsolve(ff,[Equity,SigmaE])
        return result

In [ ]:
os.chdir('new_kmv_output_1012/')

In [ ]:
for i in range(len(stockcode)):
    print(i)
    # Get debt, share, bps data
    debtpart = debtuse[debtuse['ts_code'] == stockcode[i]].reset_index().drop(['index'], axis=1)
    sharepart = shareuse[shareuse['Stkcd'] == stockcode[i]].reset_index().drop(['index'], axis=1)
    bpspart = bpsuse[bpsuse['ts_code'] == stockcode[i]].reset_index().drop(['index'], axis=1)
    output = []
    for j in calendardate:
        print(j)
        # Get DP and D values
        debtdata = debtpart[debtpart['CalendarMonth'] == j]
        if (debtdata is None) == True:
            DP = None
            D = None
        else:
            if debtdata.empty == True:
                DP = None
                D = None
            else:
                if (debtdata.iloc[0,5] == nan) or (debtdata.iloc[0,5] == None) or (debtdata.iloc[0,6] == nan) or (debtdata.iloc[0,6] == None) or (debtdata.iloc[0,7] == nan) or (debtdata.iloc[0,7] == None):
                    DP = None
                    D = None
                else:
                    DP = debtdata.iloc[0,5] + 0.5*debtdata.iloc[0,6]
                    D = debtdata.iloc[0,7]
        
        # Get bps data
        bpsdata = bpspart[bpspart['CalendarMonth'] == j]
        if (bpsdata is None) == True:
            bpsvalue = None
        else:
            if bpsdata.empty == True:
                bpsvalue = None
            else:
                if (bpsdata.iloc[0,3] == nan) or (bpsdata.iloc[0,3] == None):
                    bpsvalue = None
                else:
                    bpsvalue = bpsdata.iloc[0,3]
        
        # Get share and non-share data
        sharedata = sharepart[sharepart['CalendarMonth'] == j]
        if (sharedata is None) == True:
            tradeshare = None
            nontradeshare = None
        else:
            if sharedata.empty == True:
                tradeshare = None
                nontradeshare = None
            else:
                if (sharedata.iloc[0,2] == nan) or (sharedata.iloc[0,2] == None) or (sharedata.iloc[0,3] == nan) or (sharedata.iloc[0,3] == None):
                    tradeshare = None
                    nontradeshare = None
                else:
                    tradeshare = sharedata.iloc[0,3]
                    nontradeshare = sharedata.iloc[0,2] - sharedata.iloc[0,3]
        
        # Get adjust stock price
        adjuststock = ts.pro_bar(ts_code=stockcode[i], freq='W', adj='qfq', start_date=str(getMonthLast(int(j))[4]), end_date=str(getMonthLast(int(j))[0]))
        if (adjuststock is None) == True:
            meanprice = None
            sumsquare = None
            squaresum = None
        else:
            if adjuststock.empty == True:
                meanprice = None
                sumsquare = None
                squaresum = None
            else:
                meanprice = np.mean(np.array(adjuststock['close'].tolist()))
                sumsquare = pow(np.sum(np.log(np.array(adjuststock['close'].tolist())/np.array(adjuststock['pre_close'].tolist()))),2)
                squaresum = np.sum(np.square(np.log(np.array(adjuststock['close'].tolist())/np.array(adjuststock['pre_close'].tolist()))))
        
        # Calculate Equity
        if (meanprice == None) or (tradeshare == None) or (nontradeshare == None) or (bpsvalue == None):
            equity = None
        else:
            equity = meanprice*tradeshare + bpsvalue*nontradeshare
        
        # Calculate sigmae
        n = 52
        sigmae = sigmaE(n, squaresum, sumsquare)
        
        # Get rf
        rfdata = rfuse[rfuse['CalendarMonth'] == j]
        r = rfdata.iloc[0,1]
        
        # Use function to calculate V and sigmaV
        if (equity == None) or (sigmae == None) or (D == None) or (DP == None):
            output.append(pd.DataFrame([j, None, None, None, None, None, None, None]).T)
        else:
            result = f(equity, sigmae, D, r, 1)
            if (result[0] == 0) or (result[1] == 0):
                output.append(pd.DataFrame([j, None, None, None, None, None, None, None]).T)
            else:
                distance = (result[0] - DP)/(result[0]*result[1])
                probability = 1 - N(distance)
                output.append(pd.DataFrame([j, distance, probability, equity, sigmae, result[0], result[1], DP]).T)
    dfoutput = pd.concat(output)
    dfoutput.columns = ['CalendarMonth', 'default_distance', 'default_probability', 'Equity','SigmaE','V','SigmaV','DP50']
    dfoutput.to_csv(stockcode[i]+'.csv', index=False, encoding='utf_8_sig')

In [ ]:
os.chdir('../')